In [2]:
import os
import sys
sys.path.insert(1,'/home/bwhiteak/ChaosESN/ESN_utils/')
from sklearn.metrics import mean_squared_error
import numpy as np
import rc_analysis as rca

In [3]:
def cosine_distance(signal,pred):
    av = signal[TRAINLENGTH:TRAINLENGTH+TEST].reshape(TEST,1)
    av = av - np.mean(av)
    bv = np.squeeze(pred[TRAINLENGTH:TRAINLENGTH+TEST]).reshape(TEST,1)
    bv = bv-np.mean(bv)
    num = np.squeeze(np.dot(av.T,bv))
    den = np.linalg.norm(av)*np.linalg.norm(bv)
    cosine_similarity = num/den
    return 1 - cosine_similarity

In [11]:
# Get target and predictions
TRAINLENGTH = 14000 # For MG 19700
TEST = 1000         # for MG 300
SIGNUM = 0
# Load target signal
sig_str = ['X','Z','MG','L3D']
target_dir = '../Targets'
target_signals = ['Xsignal.npy',
                  'Zsignal.npy',
                  'MGsignal.npy',
                  'L3Dsignal.npy']
fpath = os.path.join(target_dir,target_signals[SIGNUM])
target = np.load(fpath, allow_pickle=True)
# Find range for NRMSE
MINMAX = np.max(target[:TRAINLENGTH+TEST]) - \
         np.min(target[:TRAINLENGTH+TEST]) 
print(f'Target {target_signals[SIGNUM]} shape is {target.shape}')

# Load in model predictions
model_dir = ['../X_Saved_Models',
             '../Z_Saved_Models',
             '../MG_Saved_Models',
             '../L3D_Saved_Models']
preds_dir = 'best/Preds'
sizes = [10,20,30,40,50,100,200,300,400,500,600,700,800,900,1000]
error_arr = np.zeros(len(sizes))
cd_arr = np.zeros(len(sizes))

for i in range(len(sizes)):
    pred_str  = f'{sig_str[SIGNUM]}_{sizes[i]}_Preds.npy'
    fppath = os.path.join(model_dir[SIGNUM],preds_dir,pred_str)
    try:
        loaded_pred = np.load(fppath, allow_pickle=True)
    except:
        print('Failed on... ', fppath)
        error_arr[i] = np.NAN
        continue
    prediction = np.squeeze(loaded_pred)
    if(SIGNUM != 3):
        nrmse = rca.NRMSE(target[TRAINLENGTH:TRAINLENGTH+TEST],
                          prediction[TRAINLENGTH:TRAINLENGTH+TEST],
                          MINMAX)
        cd = cosine_distance(target,prediction)
    else:
        nrmse = np.max(mean_squared_error(target[TRAINLENGTH:TRAINLENGTH+TEST],
                                   prediction[TRAINLENGTH:TRAINLENGTH+TEST],
                                   multioutput='raw_values',
                                   squared=False)/MINMAX) # Broadcasts over array dims
        
    print(f'Size {sizes[i]} NRMSE {nrmse:.2f} Cosine Distance {cd:.2f} loss {(nrmse+cd):.2f}')
    error_arr[i] = nrmse
    cd_arr[i] = cd

sigerr_str = ['./NBoutputs/ERRmmX',
              './NBoutputs/ERRmmZ',
              './NBoutputs/ERRmmMG',
              './NBoutputs/ERRmmL3D']

sigcd_str = ['./NBoutputs/CDmmX',
              './NBoutputs/CDmmZ',
              './NBoutputs/CDmmMG',
              './NBoutputs/CDmmL3D']



np.save(sigerr_str[SIGNUM],error_arr)
np.save(sigcd_str[SIGNUM],cd_arr)

Target Xsignal.npy shape is (16000,)
Size 10 NRMSE 0.22 Cosine Distance 0.60 loss 0.83
Size 20 NRMSE 0.21 Cosine Distance 0.53 loss 0.74
Size 30 NRMSE 0.22 Cosine Distance 0.53 loss 0.75
Size 40 NRMSE 0.21 Cosine Distance 0.49 loss 0.70
Size 50 NRMSE 0.22 Cosine Distance 0.52 loss 0.73
Size 100 NRMSE 0.22 Cosine Distance 0.57 loss 0.79
Size 200 NRMSE 0.20 Cosine Distance 0.47 loss 0.67
Size 300 NRMSE 0.22 Cosine Distance 0.60 loss 0.82
Size 400 NRMSE 0.21 Cosine Distance 0.47 loss 0.68
Size 500 NRMSE 0.22 Cosine Distance 0.54 loss 0.75
Size 600 NRMSE 0.21 Cosine Distance 0.50 loss 0.71
Size 700 NRMSE 0.21 Cosine Distance 0.55 loss 0.76
Size 800 NRMSE 0.23 Cosine Distance 0.59 loss 0.81
Size 900 NRMSE 0.24 Cosine Distance 0.65 loss 0.89
Size 1000 NRMSE 0.18 Cosine Distance 0.37 loss 0.55
